In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import geoviews as gv
import hvplot.pandas
hv.extension('bokeh')
gv.extension('bokeh')

### read

In [2]:
gdf = gpd.read_file('kui_leftbank_with_elevation.geojson')
Z = gdf['Elevation'].values
kp = gdf['距離標名'].values
kpL = kp.astype(float)

In [3]:
df = pd.read_csv('xy_h.out', skiprows=10, sep="\\s+", names=['x', 'y', 'hin','hout'])

### location map

In [4]:
back = gv.WMTS('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name="GoogleMapsImagery" )
g1 = gdf.to_crs(epsg=3857).hvplot(height=300, width=600, hover_cols=['距離標名'], use_index=False, size=2, xlabel='Longitude', ylabel='Latitude')*back
g1

:Overlay
   .Points.I :Points   [x,y]   (距離標名)
   .WMTS.I   :WMTS   [Longitude,Latitude]

In [6]:
hv.save(g1, 'kui_leftbank_elevation.html')

### longitudinal profile

In [7]:
hout = df['hout'].values

g21 = hv.Curve((kpL, Z), '距離標名', 'Elevation', label='JGD2011') \
*hv.Curve((kpL, hout), '距離標名', 'Elevation', label='JGD2024')

g22 = hv.Curve((kpL, hout - Z), '距離標名', 'diff_Elevation')

g21o = g21.opts(width=600, height=400, show_grid=True, title="左岸堤防高", legend_position='bottom_right')
g22o = g22.opts(width=600, height=200, show_grid=True, title="差分：JGD2024 - JGD2011")
g2 = (g21o + g22o).cols(1)
g2

:Layout
   .Overlay.I :Overlay
      .Curve.JGD2011 :Curve   [距離標名]   (Elevation)
      .Curve.JGD2024 :Curve   [距離標名]   (Elevation)
   .Curve.I   :Curve   [距離標名]   (diff_Elevation)

In [8]:
hv.save(g2, 'kui_leftbank_elevation_plot.html')